In [1]:
from cameo import load_model
from cameo import fba

import escher
from escher import Builder

import cobra

import pandas as pd
import json

In [2]:
# load Chromobacterium model
model = load_model('../results/iDB858_curated_phase1.xml')
model

Name,M_iDB858
Memory address,0x07fc6987f7f40
Number of metabolites,1003
Number of reactions,1330
Number of groups,84
Objective expression,1.0*biomass1.12 - 1.0*biomass1.12_reverse_e00c6
Compartments,Cytoplasm


In [3]:
# load json map
with open ("../results/edited_map_reaction_curated_phase1.json", "r") as f:
    data = json.loads(f.read())

In [4]:
# load reaction mapping
df_matching_reactions = pd.read_csv("../tables/reaction_map.csv", index_col=0)
mask = df_matching_reactions["name (in model)"].notna() # drop null values
df_matching_reactions = df_matching_reactions[mask]
df_matching_reactions.head(2)

,name,name (in model),bigg_id,bigg_id (in model),gene_reaction_rule,genes,metabolites,metabolites (in model)
0,Phosphoglycerate kinase,ATP 3-phospho-D-glycerate 1-phosphotransferase,PGK,rxn01100_c,b2926,"[{'bigg_id': 'b2926', 'name': 'pgk'}]","[{'coefficient': -1, 'bigg_id': '3pg_c'}, {'co...","[{'coefficient': -1.0, 'bigg_id': 'h_c'}, {'co..."
3,Fructose-bisphosphatase,D-Fructose-1 6-bisphosphate 1-phosphohydrolase,FBP,rxn00549_c,b3925 or b4232,"[{'bigg_id': 'b4232', 'name': 'fbp'}, {'bigg_i...","[{'coefficient': -1, 'bigg_id': 'fdp_c'}, {'co...","[{'coefficient': -1.0, 'bigg_id': 'h2o_c'}, {'..."


In [5]:
df_matching_reactions.bigg_id.values

array(['PGK', 'FBP', 'PYK', 'PFK', 'PTAr', 'ME1', 'ACKr', 'FUMt2_2',
       'SUCCt3', 'GAPD', 'SUCCt2_2', 'GLUDy', 'FORti', 'FRUpts2', 'SUCDi',
       'D_LACt2', 'BIOMASS_Ecoli_core_w_GAM', 'DDPA', 'DHQS', 'SHKK',
       'PSCVT', 'CHORS', 'PHEt2r', 'rxnvio6', 'DF_dvio', 'rxnvio9',
       'DF_vio'], dtype=object)

In [6]:
map_nodes = {}
for n in data[1]["reactions"]:   
    query = data[1]["reactions"][n]["bigg_id"]
    if query in df_matching_reactions.bigg_id.values:
        map_nodes[query] = n

In [7]:
for idx in df_matching_reactions.index:
    query_model = df_matching_reactions.loc[idx, "bigg_id (in model)"]
    target_map = df_matching_reactions.loc[idx, "bigg_id"]
    if not query_model == "biomass1.12": #except biomass
        print(query_model, "-->", target_map)
        model.reactions.get_by_id(query_model).id = target_map
        data[1]["reactions"][map_nodes[target_map]]["name"] = df_matching_reactions.loc[idx, "name (in model)"]
    else:
        data[1]["reactions"][map_nodes[target_map]]["bigg_id"] = query_model # except for biomass

with open('../results/edited_map_reaction_curated_phase2.json', 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

rxn01100_c --> PGK
rxn00549_c --> FBP
rxn00148_c --> PYK
rxn00545_c --> PFK
rxn00173_c --> PTAr
rDB00157_c --> ME1
rxn00225_c --> ACKr
rxn10152_c --> FUMt2_2
rxn05654_c --> SUCCt3
rxn00781_c --> GAPD
rxn10154_c --> SUCCt2_2
GLUSy --> GLUDy
FORt2 --> FORti
rxn05560_c --> FRUpts2
FRD7 --> SUCDi
EX_lac__D_e --> D_LACt2
rxn01332_c --> DDPA
rxn02212_c --> DHQS
rxn01739_c --> SHKK
rxn02476_c --> PSCVT
rxn01255_c --> CHORS
rDB00086_c --> PHEt2r
rxnvio2 --> rxnvio6
rDB00101_c --> DF_dvio
rDB00098_c --> rxnvio9
rDB00099_c --> DF_vio


In [8]:
# simulasi model
s = fba(model)
flux = s.fluxes.to_frame()
s.objective_value

0.2515998481207547

In [9]:
# map from author
builder_scratch = Builder(
    map_json = "../results/edited_map_reaction_curated_phase2.json",
    model = model,
    reaction_data = s.fluxes
)
builder_scratch

Builder(reaction_data={'rxn12008_c': 0.00233685938934557, 'rxn00541_c': 0.0, 'ACKr': 0.0, 'rxn01674_c': 0.0, '…

In [10]:
cobra.io.write_sbml_model(model, "../results/iDB858_curated_phase2.xml")